In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
%%hive
DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;

DROP TABLE IF EXISTS t0;
OK
Time taken: 10.217 seconds
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
OK
Time taken: 0.701 seconds
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;
Loading data to table default.t0
OK
Time taken: 1.342 seconds


In [5]:
%%hive 
SELECT * FROM t0 LIMIT 5;

SELECT * FROM t0 LIMIT 5;
OK
E	["b","g","f"]	{"jjj":3,"bbb":0,"ddd":9,"ggg":8,"hhh":2}
A	["a","f","c"]	{"ccc":2,"ddd":0,"aaa":3,"hhh":9}
B	["f","e","a","c"]	{"ddd":2,"ggg":5,"ccc":6,"jjj":1}
A	["a","b"]	{"hhh":9,"iii":5,"eee":7,"bbb":1}
C	["f","g","d","a"]	{"iii":6,"ddd":5,"eee":4,"jjj":3}
Time taken: 0.708 seconds, Fetched: 5 row(s)


In [14]:
%%hive 
DROP TABLE IF EXISTS dataT; 
CREATE TABLE dataT (c2 ARRAY<CHAR(1)>, registro STRING);
INSERT OVERWRITE TABLE dataT
SELECT
    c2,
    clave
FROM
    t0
LATERAL VIEW
    explode(c3) t0 AS clave,valor;

DROP TABLE IF EXISTS dataT; 
OK
Time taken: 0.1 seconds
CREATE TABLE dataT (c2 ARRAY<CHAR(1)>, registro STRING);
OK
Time taken: 0.051 seconds
INSERT OVERWRITE TABLE dataT
SELECT
    c2,
    clave
FROM
    t0
LATERAL VIEW
    explode(c3) t0 AS clave,valor;
Query ID = root_20200228163258_e5b89ba2-0c7d-4968-b7d7-224efd05baf1
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582902829190_0022, Tracking URL = http://5c70801abc21:8088/proxy/application_1582902829190_0022/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582902829190_0022
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-28 16:33:07,066 Stage-1 map = 0%,  reduce = 0%
2020-02-28 16:33:12,723 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.62 sec
MapReduce Total cumulative CPU time: 1 seconds 620 msec
Ended Job = job_1582902829190_0022
Stage-4 is selected by condition resolver.
Stage-3 is filtered ou

In [16]:
%%hive 
SELECT * FROM dataT LIMIT 5;

SELECT * FROM dataT LIMIT 5;
OK
["b","g","f"]	jjj
["b","g","f"]	bbb
["b","g","f"]	ddd
["b","g","f"]	ggg
["b","g","f"]	hhh
Time taken: 0.163 seconds, Fetched: 5 row(s)


In [20]:
%%hive 
DROP TABLE IF EXISTS dataTable; 
CREATE TABLE dataTable (letra STRING, registro STRING);
INSERT OVERWRITE TABLE dataTable
SELECT
    letra,
    registro
FROM
    dataT
LATERAL VIEW
    explode(c2) dataT AS letra
ORDER BY
    letra,
    registro;

DROP TABLE IF EXISTS dataTable; 
OK
Time taken: 0.081 seconds
CREATE TABLE dataTable (letra STRING, registro STRING);
OK
Time taken: 0.049 seconds
INSERT OVERWRITE TABLE dataTable
SELECT
    letra,
    registro
FROM
    dataT
LATERAL VIEW
    explode(c2) dataT AS letra
ORDER BY
    letra,
    registro;
Query ID = root_20200228163805_5cfcdf39-288f-49c0-bf0d-6ce490454d90
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582902829190_0025, Tracking URL = http://5c70801abc21:8088/proxy/application_1582902829190_0025/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582902829190_0025
Hadoop job information for Stage-1: number of mappers:

In [42]:
%%hive 
SELECT * FROM dataTable LIMIT 15;

SELECT * FROM dataTable LIMIT 15;
OK
a	aaa
a	aaa
a	aaa
a	aaa
a	aaa
a	bbb
a	bbb
a	bbb
a	bbb
a	bbb
a	bbb
a	bbb
a	ccc
a	ccc
a	ccc
Time taken: 13.221 seconds, Fetched: 15 row(s)


In [30]:
%%hive
DROP TABLE IF EXISTS word_counts;
CREATE TABLE word_counts
AS
    SELECT word2, word1, count(1) AS count
    FROM
        (SELECT letra AS word1, registro AS word2 FROM dataTable) w
GROUP BY
    word1, word2
ORDER BY
    word1;

DROP TABLE IF EXISTS word_counts;
OK
Time taken: 0.06 seconds
CREATE TABLE word_counts
AS
    SELECT word2, word1, count(1) AS count
    FROM
        (SELECT letra AS word1, registro AS word2 FROM dataTable) w
GROUP BY
    word1, word2
ORDER BY
    word1;
Query ID = root_20200228165004_2cbaed11-010b-4f2c-bf22-b22baa8768ac
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582902829190_0030, Tracking URL = http://5c70801abc21:8088/proxy/application_1582902829190_0030/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582902829190_0030
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
202

In [34]:
%%hive
SELECT word1, word2, count FROM word_counts ORDER BY word1, word2;

SELECT word1, word2, count FROM word_counts ORDER BY word1, word2;
Query ID = root_20200228165323_0eb79880-68b5-4a01-a07d-dc0a3184ef08
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582902829190_0033, Tracking URL = http://5c70801abc21:8088/proxy/application_1582902829190_0033/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582902829190_0033
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-28 16:53:31,763 Stage-1 map = 0%,  reduce = 0%
2020-02-28 16:53:37,370 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 1.54 sec
2020-02-28 16:53:42,839 Stage-1 map = 100%,  reduce = 100%, Cumulative CPU

In [47]:
%hive_quit

In [48]:
## Copia Archivos a sistema HDFS
##
## Se usan un directorio temporal en el HDFS. La siguiente
## instrucción muestra el contenido del dicho directorio
##
!hdfs dfs -ls /tmp

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-28 15:14 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 15:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-28 16:20 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-28 15:33 /tmp/wordcount


In [49]:
## Crea la carpeta wordcount en el hdfs
##
!hdfs dfs -mkdir /tmp/wordcount

mkdir: `/tmp/wordcount': File exists


In [50]:
## Verifica la creación de la carpeta
##
!hdfs dfs -ls /tmp/

Found 4 items
drwxrwx---   - root supergroup          0 2020-02-28 15:14 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-28 15:19 /tmp/hive
drwxrwxrwx   - root supergroup          0 2020-02-28 16:20 /tmp/output
drwxr-xr-x   - root supergroup          0 2020-02-28 15:33 /tmp/wordcount


In [51]:
## Copia los archvios del directorio local wordcount/
## al directorio /tmp/wordcount/ en el hdfs
##
!hdfs dfs -copyFromLocal 'data.tsv'  /tmp/wordcount/

copyFromLocal: `/tmp/wordcount/data.tsv': File exists


In [52]:
##
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

Found 1 items
-rw-r--r--   1 root supergroup       1394 2020-02-28 15:33 /tmp/wordcount/data.tsv


In [53]:
####################################################################################################################
## Generación del script y ajuste del código

In [54]:
%%writefile wordcount.hql

DROP TABLE IF EXISTS t0;
CREATE TABLE t0 (
    c1 STRING,
    c2 ARRAY<CHAR(1)>, 
    c3 MAP<STRING, INT>
    )
    ROW FORMAT DELIMITED 
        FIELDS TERMINATED BY '\t'
        COLLECTION ITEMS TERMINATED BY ','
        MAP KEYS TERMINATED BY '#'
        LINES TERMINATED BY '\n';
LOAD DATA LOCAL INPATH 'data.tsv' INTO TABLE t0;


DROP TABLE IF EXISTS dataT; 
CREATE TABLE dataT (c2 ARRAY<CHAR(1)>, registro STRING);
INSERT OVERWRITE TABLE dataT
SELECT
    c2,
    clave
FROM
    t0
LATERAL VIEW
    explode(c3) t0 AS clave,valor;
    
    
DROP TABLE IF EXISTS dataTable; 
CREATE TABLE dataTable (letra STRING, registro STRING);
INSERT OVERWRITE TABLE dataTable
SELECT
    letra,
    registro
FROM
    dataT
LATERAL VIEW
    explode(c2) dataT AS letra
ORDER BY
    letra,
    registro;
    
DROP TABLE IF EXISTS word_counts;
CREATE TABLE word_counts
AS
    SELECT word2, word1, count(1) AS count
    FROM
        (SELECT letra AS word1, registro AS word2 FROM dataTable) w
GROUP BY
    word1, word2
ORDER BY
    word1;
    
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

SELECT word1, word2, count FROM word_counts ORDER BY word1, word2;

Overwriting wordcount.hql


In [55]:
## Ejecucion
!hive -S -e 'source wordcount.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [56]:
### Copia de los resultados a la maquina local (vagrant)
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [57]:
!cat output/000000_0 | head

a,aaa,5
a,bbb,7
a,ccc,13
a,ddd,13
a,eee,7
a,fff,10
a,ggg,8
a,hhh,8
a,iii,7
a,jjj,10
